In [1]:
import folium 
import json
import pandas as pd
import pathlib
import fiona
import folium
import os
from folium.plugins import MarkerCluster
from folium.plugins import Draw
from shapely.geometry import LineString
import geopandas as gpd
import pickle

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
ports_df = pd.read_csv('ports_lat_long.csv',encoding = "ISO-8859-1")

In [4]:
geo_data = gpd.read_file('attributed_ports_updated.shx')

In [5]:
geo_data.head()

,geometry
0,POINT (54.36667 24.46667)
1,POINT (52.73333 24.11667)
2,POINT (56.35000 25.33333)
3,POINT (55.36667 25.35000)
4,POINT (54.66667 24.83333)


In [6]:
df = pd.read_csv('ports.csv',encoding = "ISO-8859-1")

In [7]:
df.head()

,Port,Latitude,Longitude
0,Abu Dhabi,54.366667,24.466667
1,Ar Ruways,52.733333,24.116667
2,Khor al Fakkan,56.350000,25.333333
3,Mina Khalid,55.366667,25.350000
4,Mina Khalifa\/Abu Dhabi,54.666667,24.833333


In [8]:
merged = pd.concat([geo_data, df], axis=1)

In [9]:
merged.head()

,geometry,Port,Latitude,Longitude
0,POINT (54.36667 24.46667),Abu Dhabi,54.366667,24.466667
1,POINT (52.73333 24.11667),Ar Ruways,52.733333,24.116667
2,POINT (56.35000 25.33333),Khor al Fakkan,56.350000,25.333333
3,POINT (55.36667 25.35000),Mina Khalid,55.366667,25.350000
4,POINT (54.66667 24.83333),Mina Khalifa\/Abu Dhabi,54.666667,24.833333


In [10]:
merged.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   geometry   400 non-null    geometry
 1   Port       400 non-null    object  
 2   Latitude   400 non-null    float64 
 3   Longitude  400 non-null    float64 
dtypes: float64(2), geometry(1), object(1)
memory usage: 12.6+ KB


In [11]:
#had to switch lat and long in order for it to map location correctly.....
gdf2 = gpd.GeoDataFrame(ports_df, geometry=gpd.points_from_xy(ports_df.Latitude, ports_df.Longitude))
gdf2.columns=['port','lat','lng','loc']
gdf2.head()

,port,lat,lng,loc
0,Abu Dhabi,24.466667,54.366667,POINT (54.36667 24.46667)
1,Ar Ruways,24.116667,52.733333,POINT (52.73333 24.11667)
2,Khor al Fakkan,25.333333,56.350000,POINT (56.35000 25.33333)
3,Mina Khalid,25.350000,55.366667,POINT (55.36667 25.35000)
4,Mina Khalifa\/Abu Dhabi,24.833333,54.666667,POINT (54.66667 24.83333)


In [12]:
drc_df = pd.read_csv('DRC-Kenya-Malaysia.csv')

In [13]:
drc_df.head()

,country,event_date,event_type,latitude,longitude,fatalities
0,Nigeria,31-Mar-21,Violence against civilians,7.0413,3.3547,0
1,Democratic Republic of Congo,20-Mar-21,Protests,-0.3100,18.9653,0
2,Democratic Republic of Congo,6-Mar-21,Protests,-10.7167,25.4725,0
3,Nigeria,2-Mar-21,Violence against civilians,7.6841,5.3867,0
4,Democratic Republic of Congo,7-Feb-21,Violence against civilians,1.3601,29.0354,1


In [14]:
m = folium.Map(location=[0, 0], tiles="cartodbdark_matter", zoom_start=3)
file = 'data_maritime_route.geojson'
file2 = 'data_maritime_route_2.geojson'

marker_cluster2 = MarkerCluster(name='Protest Popup', show=False)
for i in range(0,len(drc_df)):
    marker2 = folium.Marker([drc_df.iloc[i]['latitude'],drc_df.iloc[i]['longitude']])
    popup2='Cntry:{}<br>Event_date:{}<br>Fatalities:{}<br>Event_type:{}<br>Lat:{}<br>Long:{}'.format(drc_df.iloc[i]['country'],drc_df.iloc[i]['event_date'],drc_df.iloc[i]['fatalities'],drc_df.iloc[i]['event_type'],drc_df.iloc[i]['latitude'],drc_df.iloc[i]['longitude'])
    folium.Popup(popup2).add_to(marker2)
    marker_cluster2.add_child(marker2)

marker_cluster = MarkerCluster(name='Port popup', show=False)
#creating popup data along with plotting ports
for i in range(0,len(merged)):  
    marker = folium.Marker([gdf2.iloc[i]['lat'],gdf2.iloc[i]['lng']])
    popup='Port:{}<br>Lat:{}<br>Long:{}<br>'.format(gdf2.iloc[i]['port'],gdf2.iloc[i]['lat'],gdf2.iloc[i]['lng'])
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)
                                
#adding the ability to draw and export geojson files    
draw = Draw(export=True,filename='data.geojson')
draw.add_to(m)

#adding the maritime route geojson file
folium.GeoJson(json.load(open(file)),name='Maritime_route', show=False, style_function= lambda feature: {'color':'blue'}).add_to(m)

folium.GeoJson(json.load(open(file2)),name='Maritime_route2', show=False, style_function= lambda feature: {'color':'blue'}).add_to(m)

folium.GeoJson(json.load(open('BCIM.txt')),name='Bangladesh-China-India-Myanmar Economic Corridor', show=False, style_function= lambda feature: {'color':'red'}).add_to(m)

folium.GeoJson(json.load(open('CCAWEC.txt')),name='China-Central Asia-Western Asia Economic Corridor', show=False, style_function= lambda feature: {'color':'yellow'}).add_to(m)

folium.GeoJson(json.load(open('CICPEC.txt')),name='China-Indochina Peninsula Economic Corridor', show=False, style_function= lambda feature: {'color':'white'}).add_to(m)

folium.GeoJson(json.load(open('CMRECcompleted.txt')),name='China-Mongolia-Russia Economic Corridor, Completed', show=False, style_function= lambda feature: {'color':'purple'}).add_to(m)

folium.GeoJson(json.load(open('CMRECplanned.txt')),name='China-Mongolia-Russia Economic Corridor, Planned', show=False,style_function= lambda feature: {'color':'green'}).add_to(m)

folium.GeoJson(json.load(open('nelbec.txt')),name='New Eurasian Land Bridge Economic Corridor', show=False, style_function= lambda feature: {'color':'violet'}).add_to(m)

folium.GeoJson(json.load(open('CPECcompleted.txt')),name='China-Pakistan Economic Corridor, Completed', show=False, style_function= lambda feature: {'color':'orange'}).add_to(m)

folium.GeoJson(json.load(open('CPECplanned.txt')),name='China-Pakistan Economic Corridor, Planned', show=False, style_function= lambda feature: {'color':'brown'}).add_to(m)


marker_cluster.add_to(m)
marker_cluster2.add_to(m)
folium.TileLayer('OpenStreetMap').add_to(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamentoner').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)
m.save("port_with_clusters.html")

In [15]:
m